# Практическое задание курса Light Auto ML. Часть 3 - Custom Solution

<details>
<summary>Описание задания</summary>

Основная задача - выбрать и решить соревнование с платформы Kaggle.com  (http://kaggle.com/) , используя два подхода:
1. Подготовить базовое решение (бейзлайн) с помощью Light Auto ML (LAMA)
2. Реализовать альтернативное решение без использования LAMA

Требования к выбору соревнования
- Можно выбрать как текущие, так и прошедшие денежные соревнования
- Другие типы соревнований необходимо согласовать с куратором курса
- Нельзя использовать простые соревнования типа Titanic

Цели проекта
- Превзойти результаты бейзлайна на LAMA
- Продемонстрировать качественный код
- Использовать стандартные подходы к организации кода (например, Pipeline)
- Провести качественный EDA
- Предоставить подробное описание и обоснование гипотез

Критерии оценки
1. Анализ целевой переменной (максимум 1 балл)
[0.5] Численный анализ:
Для регрессии: распределение таргета, поиск аномальных значений
Для классификации: распределение количества классов
[0.5] Визуализация статистик:
- Изолированный анализ
- Анализ во временном контексте

2. Анализ признаков (максимум 4 балла)
[0.5] Типизация признаков (числовые, категориальные, временные) и их распределения
[0.5] Выявление аномальных значений
[0.5] Анализ зависимостей между признаками
[0.5] Анализ пропущенных значений
[0.5] Определение важности признаков (корреляции с таргетом)
[1.0] Графическая визуализация минимум 3-х пунктов выше
[0.5] Анализ возможных преобразований и генерации новых признаков

3. Моделирование (максимум 3.5 балла)
[0.25] Обоснование стратегии разделения данных (train-test split)
Особое внимание уделить предотвращению утечки данных
[0.25] LAMA бейзлайн:
- Минимум 2 различные конфигурации
- Выбор лучшего решения
[3.0] Собственное решение (если не удалось побить LLama baseline: 3 x 1.0 балл за различные пайплайны/попытки):
- Выбор модели
- Построение пайплайна (препроцессинг, обработка пропусков, генерация признаков, отбор признаков, финальная модель/ансамбль)
- Оптимизация гиперпараметров

4. Общие требования к коду (максимум 1.5 балла)
[0.5] Чистый код:
- Оформление ноутбука
- Соответствие PEP 8
- Правильное именование переменных и функций
- Документирование функций
[0.5] Качество кода:
- Следование принципам SOLID
- Отсутствие спагетти-кода
- Обработка предупреждений и ошибок
- Логгирование
[0.5] Структура решения:
- Оформление в виде self-contained pipeline
- Использование стандартных инструментов (например, sklearn pipeline)

Итоговая оценка
Максимальный балл: 10
9-10 баллов: оценка 5А
7-8.5 баллов: оценка 4В
5-6.5 баллов: оценка 3D
Менее 5 баллов: требуется пересдача

Ожидания
Работа должна представлять собой мини-исследование с:
1) Проработкой и проверкой гипотез
2) Оценкой результатов
3) Обоснованием выбора пайплайна
4) Документированием процесса исследования

</details>

### Часть 3 - Custom Solution

In [ ]:

# импорт нужных библиотек
import pandas as pd

from sklearn.model_selection import cross_val_score, GroupKFold
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import make_scorer, mean_absolute_error
from lightgbm import LGBMRegressor, early_stopping
import optuna
import numpy as np

In [39]:
# загрузка данных
train = pd.read_csv('./data/train.csv')
test  = pd.read_csv('./data/test.csv')
sample = pd.read_csv('./data/sample_submission.csv')

print(f"Размер тренировочной выборки: {train.shape}")
print(f"Размер тестовой выборки: {test.shape}")
print(f"Размер sample выборки: {sample.shape}")

Размер тренировочной выборки: (6036000, 8)
Размер тестовой выборки: (4024000, 7)
Размер sample выборки: (4024000, 2)


##### **3.1 Генерации новых признаков**

Вспомним наш стек добавочных признаков:

**1. Lag-признаки**
- `u_in_lag_1/2/3` - предыдущие значения управляющего сигнала
- так как корреляция внутри циклов в 2-3 раза сильнее глобальной

**2. Difference признаки**
- `u_in_diff` - скорость изменения управляющего сигнала
- динамика изменений критична для временных моделей

**3. Cumulative sum**
- `u_in_cumsum` - накопленный объем воздуха
-  физический смысл - интеграл потока

**4. Взаимодействия R×C**
- `R_x_C` - произведение параметров легких
- график показал нелинейное влияние комбинаций

**5. Позиция в цикле**
- `time_position` - порядковый номер шага внутри breath_id (0-79)
- позволяет модели определять фазу дыхания независимо от абсолютного времени

In [40]:
def create_features(df):
    """
    Создание признаков на основе анализа:
    - Lag features
    - Difference
    - Cumulative sum
    - R×C interaction
    """
    df = df.copy()
    
    for lag in [1, 2, 3]:
        df[f'u_in_lag_{lag}'] = df.groupby('breath_id')['u_in'].shift(lag)

    df['u_in_diff'] = df.groupby('breath_id')['u_in'].diff()
    df['u_in_cumsum'] = df.groupby('breath_id')['u_in'].cumsum()
    df['R_x_C'] = df['R'] * df['C']
    df['time_position'] = df.groupby('breath_id').cumcount()
    
    # Заполнение NaN в lag-признаках нулями
    lag_cols = [f'u_in_lag_{i}' for i in [1, 2, 3]] + ['u_in_diff']
    df[lag_cols] = df[lag_cols].fillna(0)
    
    print(f"Created {len(lag_cols) + 3} new features")
    return df

train_fe = create_features(train)

# Список всех признаков
ORIGINAL_FEATURES = ['R', 'C', 'time_step', 'u_in', 'u_out']
NEW_FEATURES = ['u_in_lag_1', 'u_in_lag_2', 'u_in_lag_3', 
                'u_in_diff', 'u_in_cumsum', 'R_x_C', 'time_position']
ALL_FEATURES = ORIGINAL_FEATURES + NEW_FEATURES

print(f"Всего признаков: {len(ALL_FEATURES)}")
print(f"   Оригинальные: {len(ORIGINAL_FEATURES)}")
print(f"   Новые: {len(NEW_FEATURES)}")
print(f"\nНовые признаки: {NEW_FEATURES}")

Created 7 new features
Всего признаков: 12
   Оригинальные: 5
   Новые: 7

Новые признаки: ['u_in_lag_1', 'u_in_lag_2', 'u_in_lag_3', 'u_in_diff', 'u_in_cumsum', 'R_x_C', 'time_position']


In [41]:
train_fe = create_features(train)
test_fe  = create_features(test)

FEATURES = ['R','C','time_step','u_in','u_out','u_in_lag_1','u_in_lag_2','u_in_lag_3',
            'u_in_diff','u_in_cumsum','R_x_C','time_position']
TARGET = 'pressure'

Created 7 new features
Created 7 new features


### **3.2 Выбор модели и стратегия валидации**

**Почему выбор упарл на LightGBM?**
1. **Результаты LAMA**: LightGBM показал лучшие результаты в LAMA экспериментах
2. **Скорость обучения**: быстрее CatBoost, что критично для Optuna оптимизации и эффективно работает с большими данными (более 6М+ строк)
3. **Встроенная обработка категориальных признаков**: R, C, u_out

**Стратегия валидации: GroupKFold**
- **5 фолдов** по breath_id - целые циклы дыхания не пересекаются между train/val
- **Предотвращение утечки**: модель не видит другие шаги того же цикла во время валидации
- **Метрика**: MAE

**Early Stopping**
- Остановка обучения через 100 итераций без улучшения
- Защита от переобучения
- Ускорение Optuna (не обучаем плохие конфигурации до конца)

**Оптимизируемые параметры:**
- `learning_rate`: 0.01 - 0.1 (log scale) - скорость обучения
- `num_leaves`: 31 - 128 - сложность деревьев
- `subsample`: 0.6 - 1.0 - доля строк для обучения каждого дерева
- `colsample_bytree`: 0.6 - 1.0 - доля признаков для каждого дерева

**Фиксированные параметры:**
- `n_estimators`: 4000 - максимальное число деревьев (с early stopping)
- `random_state`: 42 - воспроизводимость

**Настройки Optuna:**
- 15 trials - компромисс между качеством и временем
- 1800 секунд timeout - максимум 30 минут
- Минимизация OOF MAE

##### A. Новый вариант с cross_val_score  + early_stopping + LGBMRegressor

In [ ]:
# сделаем валидационный сет исключительно для этого варианта
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(gss.split(train_fe, groups=train_fe['breath_id']))

X_train = train_fe.iloc[train_idx][FEATURES]
y_train = train_fe.iloc[train_idx][TARGET]

X_val = train_fe.iloc[val_idx][FEATURES]
y_val = train_fe.iloc[val_idx][TARGET]

groups_train = train_fe.iloc[train_idx]['breath_id']

In [ ]:
def objective(trial):
    params = {
        'n_estimators': 4000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 31, 128),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'random_state': 42,
        'n_jobs': -1
    }

    model = LGBMRegressor(**params)

    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
    gkf = GroupKFold(n_splits=5, random_state=42)

    scores = cross_val_score(
        model,
        X_train,
        y_train,
        groups=groups_train,
        cv=gkf,
        scoring=mae_scorer,
        params={
            "eval_set": [(X_val, y_val)],
            "eval_metric": "mae",
            "callbacks": [early_stopping(stopping_rounds=100)]
        },
        n_jobs=-1
    )

    return -scores.mean()


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=15, timeout=1800)

print("Best params:", study.best_params)
print("Best OOF MAE:", study.best_value)

[I 2025-12-31 00:28:50,919] A new study created in memory with name: no-name-314cb6d6-af98-4a5c-91ef-aef70410ce7d


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027778 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 3863040, number of used features: 12
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029109 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 3863040, number of used features: 12
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044030 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of da

[I 2025-12-31 00:41:27,448] Trial 0 finished with value: 0.5158440473966881 and parameters: {'learning_rate': 0.038346540556094974, 'num_leaves': 119, 'subsample': 0.7365705298918935, 'colsample_bytree': 0.7646452608285308}. Best is trial 0 with value: 0.5158440473966881.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.134639 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 3863040, number of used features: 12
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020308 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 3863040, number of used features: 12
[LightGBM] [Info] Start training from score 11.220706
[LightGBM] [Info] Start training from score 11.219441
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, 

[I 2025-12-31 00:52:01,815] Trial 1 finished with value: 0.6222257345565322 and parameters: {'learning_rate': 0.015695404994591627, 'num_leaves': 63, 'subsample': 0.6409757352945158, 'colsample_bytree': 0.6371299265500254}. Best is trial 0 with value: 0.5158440473966881.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017311 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 3863040, number of used features: 12
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022784 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 3863040, number of used features: 12
[LightGBM] [Info] Start training from score 11.220706
[LightGBM] [Info] Start training from score 11.217725
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, 

[I 2025-12-31 00:59:51,322] Trial 2 finished with value: 0.5656365151522015 and parameters: {'learning_rate': 0.03780269167918193, 'num_leaves': 50, 'subsample': 0.6716679412529537, 'colsample_bytree': 0.915971920569901}. Best is trial 0 with value: 0.5158440473966881.


Best params: {'learning_rate': 0.038346540556094974, 'num_leaves': 119, 'subsample': 0.7365705298918935, 'colsample_bytree': 0.7646452608285308}
Best OOF MAE: 0.5158440473966881


##### B. Новый вариант с cross_val_score + LGBMRegressor

In [ ]:
# def objective(trial):
#     # параметры для Optuna
#     params = {
#         'n_estimators': 4000,
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
#         'num_leaves': trial.suggest_int('num_leaves', 31, 128),
#         'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
#         'random_state': 42,
#         'n_jobs': -1
#     }
    
#     model = LGBMRegressor(**params)
#     mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
#     gkf = GroupKFold(n_splits=5, random_state=42)
    
#     # cross_val_score с группами
#     scores = cross_val_score(
#         model,
#         train_fe[FEATURES],
#         train_fe[TARGET],
#         groups=train_fe['breath_id'], # по циклам дыхания
#         cv=gkf,
#         scoring=mae_scorer,
#         n_jobs=-1
#     )
    
#     # возвращаем среднюю MAE по фолдам (минус для положительного числа)
#     return -scores.mean()


# # Run Optuna
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=15, timeout=1800)  # до 30 минут

# print("Best params:", study.best_params)
# print("Best OOF MAE:", study.best_value)

[I 2025-12-30 19:40:00,293] A new study created in memory with name: no-name-74a04cc5-e8fd-4fb0-98d4-a9da191da1aa
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_61214/235241336.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_61214/235241336.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_61214/235241336.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://git

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.266543 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 4828800, number of used features: 12
[LightGBM] [Info] Start training from score 11.223842
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.083245 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 4828800, number of used features: 12
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.113680 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points 

[I 2025-12-30 19:50:01,359] Trial 0 finished with value: 0.5633654128733779 and parameters: {'learning_rate': 0.02853590335634323, 'num_leaves': 64, 'subsample': 0.6531058513704433, 'colsample_bytree': 0.9318078793865102}. Best is trial 0 with value: 0.5633654128733779.
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_61214/235241336.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_61214/235241336.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020012 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 4828800, number of used features: 12
[LightGBM] [Info] Start training from score 11.225156
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026094 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 4828800, number of used features: 12
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.051605 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM]

[I 2025-12-30 19:59:01,387] Trial 1 finished with value: 0.5232370180050429 and parameters: {'learning_rate': 0.08496274730236524, 'num_leaves': 60, 'subsample': 0.6008482099306177, 'colsample_bytree': 0.6224201140927726}. Best is trial 1 with value: 0.5232370180050429.
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_61214/235241336.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_61214/235241336.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027725 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 4828800, number of used features: 12
[LightGBM] [Info] Start training from score 11.223842
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.033546 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 4828800, number of used features: 12
[LightGBM] [Info] Start training from score 11.225156
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.043067 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not

[I 2025-12-30 20:11:52,452] Trial 2 finished with value: 0.494887009625485 and parameters: {'learning_rate': 0.05332723976196547, 'num_leaves': 123, 'subsample': 0.8591690890817245, 'colsample_bytree': 0.7140709673483161}. Best is trial 2 with value: 0.494887009625485.


Best params: {'learning_rate': 0.05332723976196547, 'num_leaves': 123, 'subsample': 0.8591690890817245, 'colsample_bytree': 0.7140709673483161}
Best OOF MAE: 0.494887009625485


#### C. Первоночальный вариант

In [ ]:
# # Optuna
# def objective(trial):
#     params = {
#         'n_estimators': 4000,
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
#         'num_leaves': trial.suggest_int('num_leaves', 31, 128),
#         'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
#         'random_state': 42,
#         'n_jobs': -1
#     }
    
#     oof = np.zeros(len(train_fe))
#     gkf = GroupKFold(n_splits=5, random_state=42)
    
#     for tr_idx, val_idx in gkf.split(train_fe, groups=train_fe['breath_id']):
#         X_tr, X_val = train_fe.iloc[tr_idx][FEATURES], train_fe.iloc[val_idx][FEATURES]
#         y_tr, y_val = train_fe.iloc[tr_idx][TARGET], train_fe.iloc[val_idx][TARGET]
        
#         model = LGBMRegressor(**params)
#         model.fit(
#             X_tr, y_tr,
#             eval_set=[(X_val, y_val)],
#             eval_metric='mae',
#             callbacks=[early_stopping(stopping_rounds=100)]
#         )
#         oof[val_idx] = model.predict(X_val)
    
#     return mean_absolute_error(train_fe[TARGET], oof)


# # Run Optuna
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=15, timeout=1800)  # до 30 минут

# print("Best params:", study.best_params)
# print("Best OOF MAE:", study.best_value)

##### D. Новый варинат с моделью из sclearn HistGradientBoostingRegressor
так как здесь удобно использовать cross_val_score + early_stopping

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

# def objective(trial):
#     params = {
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
#         'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 31, 128),
#         'max_iter': 4000,
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 20, 100),
#         'max_depth': trial.suggest_int('max_depth', 3, 15),
#         'random_state': 42,
#         'early_stopping': True,
#         'n_iter_no_change': 100,
#         'validation_fraction': 0.2
#     }
#     mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
#     gkf = GroupKFold(n_splits=5, random_state=42)   
#     model = HistGradientBoostingRegressor(**params)

#     # cross_val_score с группами
#     scores = cross_val_score(
#         model,
#         train_fe[FEATURES],
#         train_fe[TARGET],
#         groups=train_fe['breath_id'],
#         cv=gkf,
#         scoring=mae_scorer,
#         n_jobs=-1
#     )

#     return -scores.mean()

# # Запуск Optuna
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=15, timeout=1800)

# print("Best params:", study.best_params)
# print("Best OOF MAE:", study.best_value)

[I 2025-12-31 01:59:29,104] A new study created in memory with name: no-name-e8fd9ceb-4747-439c-964c-ba62be0911a0
/Users/anastasia/.pyenv/versions/3.11.11/lib/python3.11/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/Users/anastasia/.pyenv/versions/3.11.11/lib/python3.11/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/Users/anastasia/.pyenv/versions/3.11.11/lib/python3.11/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setup

Best params: {'learning_rate': 0.03571551063552218, 'max_leaf_nodes': 76, 'min_samples_leaf': 50, 'max_depth': 8}
Best OOF MAE: 0.5520620022838456


### **3.3 Обучение финальной модели**

Используем лучшие параметры из Optuna для обучения на всех данных.

In [ ]:
# # Train final model with best params for LGBMRegressor
# best_params = study.best_params
# best_params['n_estimators'] = 4000
# best_params['random_state'] = 42
# best_params['n_jobs'] = -1

# final_model = LGBMRegressor(**best_params)
# final_model.fit(train_fe[FEATURES], train_fe[TARGET])

In [ ]:
# Train final model with best params
best_params = study.best_params
best_params['max_iter'] = 4000
best_params['random_state'] = 42

final_model = HistGradientBoostingRegressor(**best_params)
final_model.fit(train_fe[FEATURES], train_fe[TARGET])

,"loss loss: {'squared_error', 'absolute_error', 'gamma', 'poisson', 'quantile'}, default='squared_error'The loss function to use in the boosting process. Note that the""squared error"", ""gamma"" and ""poisson"" losses actually implement""half least squares loss"", ""half gamma deviance"" and ""half poissondeviance"" to simplify the computation of the gradient. Furthermore,""gamma"" and ""poisson"" losses internally use a log-link, ""gamma""requires ``y > 0`` and ""poisson"" requires ``y >= 0``.""quantile"" uses the pinball loss... versionchanged:: 0.23 Added option 'poisson'... versionchanged:: 1.1 Added option 'quantile'... versionchanged:: 1.3 Added option 'gamma'.",'squared_error'
,"quantile quantile: float, default=NoneIf loss is ""quantile"", this parameter specifies which quantile to be estimatedand must be between 0 and 1.",None
,"learning_rate learning_rate: float, default=0.1The learning rate, also known as *shrinkage*. This is used as amultiplicative factor for the leaves values. Use ``1`` for noshrinkage.",0.03571551063552218
,"max_iter max_iter: int, default=100The maximum number of iterations of the boosting process, i.e. themaximum number of trees.",4000
,"max_leaf_nodes max_leaf_nodes: int or None, default=31The maximum number of leaves for each tree. Must be strictly greaterthan 1. If None, there is no maximum limit.",76
,"max_depth max_depth: int or None, default=NoneThe maximum depth of each tree. The depth of a tree is the number ofedges to go from the root to the deepest leaf.Depth isn't constrained by default.",8
,"min_samples_leaf min_samples_leaf: int, default=20The minimum number of samples per leaf. For small datasets with lessthan a few hundred samples, it is recommended to lower this valuesince only very shallow trees would be built.",50
,"l2_regularization l2_regularization: float, default=0The L2 regularization parameter penalizing leaves with small hessians.Use ``0`` for no regularization (default).",0.0
,"max_features max_features: float, default=1.0Proportion of randomly chosen features in each and every node split.This is a form of regularization, smaller values make the trees weakerlearners and might prevent overfitting.If interaction constraints from `interaction_cst` are present, only allowedfeatures are taken into account for the subsampling... versionadded:: 1.4",1.0
,"max_bins max_bins: int, default=255The maximum number of bins to use for non-missing values. Beforetraining, each feature of the input array `X` is binned intointeger-valued bins, which allows for a much faster training stage.Features with a small number of unique values may use less than``max_bins`` bins. In addition to the ``max_bins`` bins, one more binis always reserved for missing values. Must be no larger than 255.",255
,"categorical_features categorical_features: array-like of {bool, int, str} of shape (n_features) or shape (n_categorical_features,), default='from_dtype'Indicates the categorical features.- None : no feature will be considered categorical.- boolean array-like : boolean mask indicating categorical features.- integer array-like : integer indices indicating categorical features.- str array-like: names of categorical features (assuming the training data has feature names).- `""from_dtype""`: dataframe columns with dtype ""category"" are considered to be categorical features. The input must be an object exposing a ``__dataframe__`` method such as pandas or polars DataFrames to use this feature.For each categorical feature, there must be at most `max_bins` uniquecategories. Negative values for categorical features encoded as numericdtypes are treated as missing values. All categorical values areconverted to floating point numbers. This means that categorical valuesof 1.0 and 1 are treated as the same category.Read more in the :ref:`User Guide ` and:ref:`sphx_glr_auto_examples_ensemble_plot_gradient_boosting_categorical.py`... versionadded:: 0.24.. versionchanged:: 1.2 Added support for feature names... ver

**Выводы по Feature Importance:**
- Наиболее важные признаки - это созданные FE (отмечены [FE])
- Lag-признаки (`u_in_lag_X`) оказались критичными для предсказания
- Базовые признаки (`u_in`, `u_out`, `time_step`) также играют важную роль
- Признак `R_x_C` (взаимодействие параметров легких) подтвердил свою значимость из EDA

In [53]:
# Predict on test
test_pred = final_model.predict(test_fe[FEATURES])

submission = pd.DataFrame({
    'id': test['id'],
    'pressure': test_pred
})
submission.to_csv('submissions/lgbm_fe_optuna_hgbr_submission.csv', index=False)
print("Saved submission file")
submission.head()

Saved submission file


,id,pressure
0,1,6.257462
1,2,5.938614
2,3,6.908497
3,4,7.701577
4,5,8.985268



#### **Итоговые выводы**

**Ключевые инсайты:**
1. **Feature Engineering критичен** - 7 новых признаков дали основной прирост
2. **Lag-признаки важнейшие** - корреляция внутри циклов в 2-3 раза сильнее
3. **GroupKFold необходим** - предотвращает утечку данных по breath_id

#### **Сравнение с LAMA Baseline и кастомными вариантами (A, B, C, D)**
где: 
- A: вариант с cross_val_score  + early_stopping + LGBM (NEW)
- B: вариант с cross_val_score + LGBM (NEW)
- **C: вариант со своим циклом по cross_val_score  + early_stopping + LGBM (OLD)**
- D: варинат с cross_val_score  + early_stopping + моделью из sclearn HistGradientBoostingRegressor (NEW)

| Модель | Признаки | Public | Private |
|--------|----------|--------|---------|
| **LAMA Config 5** | 12 (5+7 FE) | 0.8480 | 0.8491 |
| **Вариант A** | 12 (5+7 FE) | 0.8483 | 0.8471 |
| **Вариант B** | 12 (5+7 FE) | 0.8196 | 0.8183 |
| **Вариант C** | 12 (5+7 FE) | **0.7761** | **0.779** |
| **Вариант D** | 12 (5+7 FE) | 0.9271 | 0.9267 |

**Вывод по экспериментам**

Были протестированы несколько стратегий обучения и валидации моделей. Использование **cross_val_score** показало себя как удобный и корректный инструмент для быстрой и воспроизводимой оценки качества моделей (варианты **A, B, D**).

При этом эксперименты с LightGBM показали, что на данной задаче временных рядов с групповой структурой (breath_id) наилучшее качество достигается при явном контроле обучения на каждом фолде с передачей собственного eval_set и использованием early stopping. Такой подход позволяет модели точнее подстраиваться под динамику внутри дыхательных циклов и даёт более стабильный результат.

В итоге именно вариант с ручным циклом по фолдам (**вариант C**) показал наилучшее качество среди всех рассмотренных решений и был выбран как финальный.

#### Результаты подтверждения с Kaggle - урезанные (только лучшая lama и кастомные решения)

![Kaggle Leaderboard](picture/submissions_best.png)